In [1]:
import pandas as pd

## Model Training

In [2]:
df = pd.read_csv('./datasets/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [5]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [6]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [7]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [8]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [9]:
categorical_cols = ['cut', 'color', 'clarity']
numerical_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']

In [10]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [11]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=30)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.473767,3.220176,-0.641637,0.445161,0.398319,0.747248,-3.151612,-0.318068,0.019657
1,-0.845697,0.075806,-0.120391,-0.880410,-0.854100,-0.862659,0.873627,0.297795,-0.646663
2,0.733333,-0.201638,0.922103,0.823895,0.897472,0.848774,-1.138993,0.913659,-0.646663
3,-0.196781,0.260769,-0.641637,-0.050801,-0.010079,-0.006943,0.873627,1.529522,0.019657
4,-0.542870,-0.571564,0.400856,-0.465605,-0.436627,-0.485564,0.873627,0.913659,0.685978


In [16]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [18]:
regression.coef_

array([[ 6434.78387743,  -132.64285991,   -70.72462747, -1744.76501257,
         -471.40887345,   -70.2527634 ,    71.08211437,  -462.11164254,
          651.0562136 ]])

In [19]:
regression.intercept_

array([3968.35483093])

In [20]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [21]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'RandomForest':RandomForestRegressor(max_depth=6),
    'DecisionTree':DecisionTreeRegressor(max_depth=6)
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1020.9106955021986
MAE: 677.5652190317544
R2 score 93.63482324438802


Lasso
Model Training Performance
RMSE: 1020.8783432223119
MAE: 678.6412565975304
R2 score 93.63522665815954


Ridge
Model Training Performance
RMSE: 1020.9104624024288
MAE: 677.5922524952899
R2 score 93.63482615104982


Elasticnet
Model Training Performance
RMSE: 1539.9317261000967
MAE: 1062.6828823951396
R2 score 85.51768818889254




/Users/rachit/Desktop/Diamond-New/diamond_project_venv/lib/python3.8/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Model Training Performance
RMSE: 721.7891153992412
MAE: 404.71173585555505
R2 score 96.81832644722198


DecisionTree
Model Training Performance
RMSE: 751.4540728920541
MAE: 423.3873440163019
R2 score 96.55142364536628




In [22]:
model_list

['LinearRegression',
 'Lasso',
 'Ridge',
 'Elasticnet',
 'RandomForest',
 'DecisionTree']

In [23]:
# Create a dictionary of classifiers and their respective hyperparameter grids for tuning
classifiers = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Lasso Regression': {
        'model': Lasso(),
        'params': {
            'alpha': [0.1, 1.0, 10.0]
        }
    },
    'Ridge Regression': {
        'model': Ridge(),
        'params': {
            'alpha': [0.1, 1.0, 10.0]
        }
    },
    'ElasticNet Regression': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.1, 1.0, 10.0],
            'l1_ratio': [0.1, 0.5, 0.9]
        }
    },
}


In [24]:
from sklearn.model_selection import GridSearchCV
best_params_list = []
# Perform hyperparameter tuning and evaluation for each classifier
for name, clf_info in classifiers.items():
    model = clf_info["model"]
    params = clf_info["params"]

    grid_search = GridSearchCV(model, params, cv=5, scoring='r2',verbose=2)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Make predictions with the best model
    y_pred = best_model.predict(X_test)

    # Evaluate the model and store results
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)
    
 # Append best params and accuracy to lists
    best_params_list.append((name, best_params))
    r2_list.append((name, r2_square))

    r2_list.append(r2_square)

    print(f"Regressor: {name}")
    print(f"Best Parameters: {best_params}")
    print(f"R2-Score: {r2_square}")
    print("=" * 50)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
Regressor: Linear Regression
Best Parameters: {}
R2-Score: 0.9363482324438801
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ..........................................alpha=0.1; total time=   1.1s
[CV] END ..........................................alpha=0.1; total time=   0.9s
[CV] END ..........................................alpha=0.1; total time=   1.0s
[CV] END ..........................................alpha=0.1; total time=   1.1s
[CV] END ..........................................alpha=0.1; total time= 

In [25]:
# Print the best parameters and accuracy for each classifier
# Not performing HyperParameter tuning for Randomforest and DecisionTree because of the time comptation
print("Best Parameters for Each Classifier:")
for name, best_params in best_params_list:
    print(f"Regressor: {name}")
    print(f"Best Parameters: {best_params}")
    print("=" * 50)

Best Parameters for Each Classifier:
Regressor: Linear Regression
Best Parameters: {}
Regressor: Lasso Regression
Best Parameters: {'alpha': 1.0}
Regressor: Ridge Regression
Best Parameters: {'alpha': 0.1}
Regressor: ElasticNet Regression
Best Parameters: {'alpha': 0.1, 'l1_ratio': 0.9}
